<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/master/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Enunciado](https://www.u-cursos.cl/ingenieria/2020/1/CC3001/1/tareas/r/1_CC3001-Tarea2-Diagrama_de_estados.pdf)